In [ ]:
# Librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib import colors
pd.options.display.max_columns = None

In [ ]:
# Lectura y limpieza de datos
datos=[]
ataque=pd.read_csv('data/attacking.csv').drop(['serial','match_played'],axis=1)
intentos=pd.read_csv('data/attempts.csv').drop(['serial','match_played'],axis=1)
defenza=pd.read_csv('data/defending.csv').drop(['serial','match_played'],axis=1)
pases=pd.read_csv('data/distributon.csv').drop(['serial','match_played'],axis=1)
porteros=pd.read_csv('data/goalkeeping.csv').drop(['serial','match_played'],axis=1)
goles=pd.read_csv('data/goals.csv').drop(['serial','match_played'],axis=1)
general=pd.read_csv('data/key_stats.csv').drop(['goals','assists'],axis=1)
datos=[intentos,defenza,pases,porteros,goles,general]
df=ataque
for i in datos:
    df=pd.merge(df,i,on=['player_name','club','position'],how='outer')
df.drop(df[df['player_name']=='Reinildo'].index,inplace=True)
df.drop(df[df['player_name']=='Burak Yilmaz'].index,inplace=True)
df.drop(df[df['player_name']=='Luis Díaz'].index,inplace=True)
df.drop(df[df['player_name']=='Brobbey'].index,inplace=True)
df['g_p']=df['goals']/df['match_played'] # Cantidad de goles por partidos jugados
df['g_m']=df['minutes_played']/df['goals'] # Cantidad de minutos para que marque un gol
df['a_p']=df['assists']/df['match_played'] # Cantidad de asistencias por partidos jugados
df['a_m']=df['minutes_played']/df['assists'] # Cantidad de minutos para que de una asistencia
df.insert(36,'no_penalties',df['goals']-df['penalties']) # Cantidad de goles que no fueron de penal
df.insert(9,'tiros_porteria',(100*df['on_target'])/df['total_attempts']) # Cantidad de tiros a porteria
df.insert(11,'tiros_no_porteria',(100*df['off_target'])/df['total_attempts']) # Cantidad de tiros sin direccion de porteria
df.insert(13,'tiros_bloqueados',(100*df['blocked'])/df['total_attempts']) # Cantidad de tiros bloqueados
df.insert(27,'salvadas_p',df['saved']/df['match_played']) # Cantidad de goles salvados por partido
df.insert(29,'goles_p',df['conceded']/df['match_played']) # Cantidad de goles permitidos por partido
df.insert(31,'penales_p',df['saved_penalties']/df['match_played']) # Cantidad de penales salvados por partido
df.insert(33,'cero_goles',(df['cleansheets']*100)/df['match_played']) # Cantidad de porterias en ceros por partido
df.fillna(0,inplace=True) # Relleno los valores NaN con 0
df.loc[df['total_attempts'] == 0.0, 'tiros_porteria'] = np.nan # Eliminar los valores 0 de tiros_porteria para que no se concideren en las graficas
df.loc[df['total_attempts'] == 0.0, 'tiros_no_porteria'] = np.nan # Eliminar los valores 0 de tiros_no_porteria para que no se concideren en las graficas
df.loc[df['total_attempts'] == 0.0, 'tiros_bloqueados'] = np.nan # Eliminar los valores 0 de tiros_bloqueados para que no se concideren en las graficas
df.loc[df['goals'] == 0.0, 'g_m'] = np.nan # Eliminar los valores 0 de g_m para que no se concideren en las graficas
df.loc[df['assists'] == 0.0, 'a_m'] = np.nan # Eliminar los valores 0 de a_m para que no se concideren en las graficas
equipos=pd.read_csv('data/club_matches.csv', encoding = 'cp1252').drop(['Unnamed: 0'],axis=1).set_index('club') # Leer partidos de equipos
equipos=pd.merge(equipos,df.groupby(['club'])['goals'].sum(), on='club') # Agregar goles totales por equipo
equipos['g_m']=equipos['goals']/equipos['teams_matches'] # Cantidad de goles por partido de equipos
df=pd.merge(df,equipos['teams_matches'],on='club') # Agregar cantidad de partidos por equipo a registro de jugadores

In [ ]:
# Ver DataFrames
# df.columns
# df
# df.describe()
# equipos.columns
# equipos
# equipos.describe()

# Datos generales
Se presentan las estadisticas totales de la temporada

In [ ]:
# Obtener datos generales, se necesita para las graficas
totales=df['goals'].sum()
izquierda=round(df['left_foot'].sum()*100/totales,2)
derecha=round(df['right_foot'].sum()*100/totales,2)
cabeza=round(df['headers'].sum()*100/totales,2)
otros=round(df['others'].sum()*100/totales,2)
dentro_del_area=round(df['inside_area'].sum()*100/totales,2)
fuera_del_area=round(df['outside_areas'].sum()*100/totales,2)
# Mostrar como texto los datos generales
print('Cantidad de equipos: {}'.format(len(pd.unique(df['club']))))
print('Cantidad de jugadores: {}'.format(len(pd.unique(df['player_name']))))
print("---------------------------------------------------")
print('Asistencias totales: {}'.format(int(df['assists'].sum())))
print('Goles totales: {}'.format(totales))
print("---------------------------------------------------")
print('Goles con la izquierda: {} --> %{}'.format(int(df['left_foot'].sum()),izquierda))
print('Goles con la derecha: {} --> %{}'.format(int(df['right_foot'].sum()),derecha))
print('Goles con la cabeza: {} --> %{}'.format(int(df['headers'].sum()),cabeza))
print('Otros: {} --> %{}'.format(int(df['others'].sum()),otros))
print("---------------------------------------------------")
print('Goles dentro del area: {} --> %{}'.format(int(df['inside_area'].sum()),dentro_del_area))
print('Goles fuera del area: {} --> %{}'.format(int(df['outside_areas'].sum()),fuera_del_area))
print("---------------------------------------------------")
print('Goles de penal: {}'.format(int(df['penalties'].sum())))
print("---------------------------------------------------")
# Grafica de pastel, Tipos de goles
datos=[derecha,izquierda,cabeza,otros]
normdata=colors.Normalize(min(datos),max(datos))
colormap = cm.get_cmap("jet")
colores =colormap(normdata(datos))
separacion=[0.02,0.02,0.02,0.02]
fig, ax = plt.subplots()
ax.pie(datos,colors=colores, explode=separacion,labels=['Derecha','Izquierda','Cabeza','Otros'], autopct="%0.1f %%",labeldistance=1.1,textprops={'color':"azure"})
ax.axis('equal')
plt.title('Tipos de goles',y=1.05,fontdict={'fontsize':20,'color':'azure', 'fontweight':'bold'})
plt.show()
# Grafica de pastel, Goles dentro y fuera del area
datos=[dentro_del_area,fuera_del_area]
normdata=colors.Normalize(min(datos),max(datos))
colormap = cm.get_cmap("turbo")
colores =colormap(normdata(datos))
separacion=[0.02,0.02]
fig, ax = plt.subplots()
ax.pie(datos,colors=colores, explode=separacion,labels=['Dentro del area','Fuera del area'], autopct="%0.1f %%",labeldistance=1.1,textprops={'color':"azure"})
ax.axis('equal')
plt.title('Goles dentro y fuera del area',y=1.05,fontdict={'fontsize':20,'color':'azure', 'fontweight':'bold'})
plt.show()

# Goleadores

In [ ]:
# Totales
df[['player_name','club','match_played','goals']].sort_values('goals',ascending=False).reset_index(drop=True).query("goals>0")

In [ ]:
# Por partido
df[['player_name','club','g_p']].sort_values('g_p',ascending=False).reset_index(drop=True).query("g_p>0")

In [ ]:
# Por minutos
df[['player_name','club','g_m']].sort_values('g_m',ascending=False).reset_index(drop=True).query("g_m>0")

In [ ]:
# Minutos por gol
df[['player_name','club','m_g']].sort_values('m_g').reset_index(drop=True).query("m_g>0")

In [ ]:
# Equipo total
df.groupby(['club'])['goals'].sum().reset_index().sort_values('goals',ascending=False).reset_index(drop=True)

In [ ]:
# Posicion
df.groupby(['position'])['goals'].sum().reset_index().sort_values('goals',ascending=False).reset_index(drop=True)

In [ ]:
# Equipo y posicion
df.groupby(['club','position'])['goals'].sum().reset_index().sort_values('goals',ascending=False).reset_index(drop=True)

# Asistencias

In [ ]:
# Totales
df[['player_name','club','match_played','assists']].sort_values('assists',ascending=False).reset_index(drop=True).query("assists>0")

In [ ]:
# Por partido
df[['player_name','club','a_p']].sort_values('a_p',ascending=False).reset_index(drop=True).query("a_p>0")

In [ ]:
# Por minutos
df[['player_name','club','a_m']].sort_values('a_m',ascending=False).reset_index(drop=True).query("a_m>0")

In [ ]:
# Posicion
df.groupby(['position'])['assists'].sum().reset_index().sort_values('assists',ascending=False).reset_index(drop=True).query("assists>0")

In [ ]:
# Equipo y posicion
df.groupby(['club','position'])['assists'].sum().reset_index().sort_values('assists',ascending=False).reset_index(drop=True).query("assists>0")